# Exp020: Eliciting target grammar skills
This is an analysis for relationships between grammar skills that may be exploited for making learners produce desired output.

In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ['CACHE_DIR'] = os.environ['FAST_CACHE_DIR'].replace("%SLURM_JOB_ID%", os.getenv('SLURM_JOB_ID')) # speed up model loading

import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from tqdm.notebook import tqdm
import torch
import pickle
#from scipy.stats import norm
import scipy.stats as sps

import sys
sys.path.append(f'../source')
import data
import evaluation
import helpers
import models
import importlib
importlib.reload(helpers)

[nltk_data] Downloading package punkt to
[nltk_data]     /scratch/tmp.58179322.dglandorf...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /scratch/tmp.58179322.dglandorf...
[nltk_data]   Package punkt is already up-to-date!


<module 'helpers' from '/cluster/home/dglandorf/grammarctg/experiments/../source/helpers.py'>

Load dialogs with their turns that we are interested in.

In [2]:
dialogs = data.get_dialog_data()
turns = helpers.flatten_list_of_lists([d[0] for d in dialogs])

The attention defines which other turns are of interest in this paradigm. Attention is only paid to next turns of the interlocutur.

In [13]:
n_turns = [len(d[0]) for d in dialogs]
turn_attention = []
n_resp = 2
idx = 0
for length in n_turns:
    for i in range(0, length):
        attention = []
        for k in range(i, min(length-1, i+(2*n_resp-1)), 2): # only attend to next n response
            attention += [idx+k+1]
        turn_attention.append(attention)
    idx += length

Let's classify the grammar in all turns.

In [43]:
skills = helpers.get_high_conf_classifiers()
classifiers = {nr: models.load_classifier(nr, "corpus_training") for nr in skills}
classified_file = '../data/turn_classification.pkl'
primed_file ='../data/prime_stats.json'

In [15]:
max_turns = int(1e10)
sentences = [(idx, sentence) for idx, turn in tqdm(enumerate(turns[:max_turns]), total=len(turns), desc="Sentence tokenization") for sentence in data.sent_tokenize(turn)]
indices, sents = [s[0] for s in sentences], [s[1] for s in sentences]

Sentence tokenization:   0%|          | 0/710640 [00:00<?, ?it/s]

In [12]:
tokenized_inputs = models.bert_tokenizer(sents, return_tensors='pt', max_length=64, padding='max_length', truncation=True)
dataset = TensorDataset(tokenized_inputs['input_ids'], tokenized_inputs['attention_mask'])
dataloader = DataLoader(dataset, batch_size=512, shuffle=False)

clf_hits = {nr: [] for nr in skills}
for input_ids, attention_mask in tqdm(dataloader, desc="Grammar classification"):    
    input_ids, attention_mask = input_ids.to(models.device), attention_mask.to(models.device)
    with torch.no_grad():
        encoded_inputs = models.bert_encoder(input_ids, attention_mask) # encoding is the same for all classifiers
        x = torch.cat(encoded_inputs.hidden_states, dim=-1)
        for nr, clf in classifiers.items():
            max_values, _ = clf.forward_bert(x, attention_mask)
            clf_hits[nr] += (max_values>0.5).cpu().tolist()

with open(classified_file, 'wb') as f:
    pickle.dump(clf_hits, f)

Sentence tokenization:   0%|          | 0/710640 [00:00<?, ?it/s]

Grammar classification:   0%|          | 0/2371 [00:00<?, ?it/s]

In [16]:
with open(classified_file, 'rb') as f:
    clf_hits = pickle.load(f)

Transform the classification into one big indicator matrix

In [17]:
hits = np.zeros([sum(n_turns), len(skills)], dtype=bool)
for idx, (nr, clf_hit) in enumerate(clf_hits.items()):
    hit_indices = np.array(indices)[clf_hit]
    hits[hit_indices,idx] = 1

Calculate base rates

In [18]:
base_rate = hits.sum(axis=0)/min(max_turns, len(turns))
base_rate

array([0.0002941 , 0.01075509, 0.0044284 , 0.00390352, 0.00116937,
       0.00012946, 0.00939013, 0.0122453 , 0.05582433, 0.00396825,
       0.00505882, 0.00543876, 0.00704435, 0.00227541, 0.00202071,
       0.00289035, 0.0016844 , 0.00027159, 0.00093296, 0.00075144,
       0.00412023, 0.00024344, 0.04089835, 0.04483001, 0.01540724,
       0.01069177, 0.027077  , 0.01082123, 0.02725571, 0.0012932 ,
       0.00024204])

Do a significance test for a changed proportion of target skills after prime skills.

In [19]:
p_val = lambda z: (1 - norm.cdf(z)) # one-sided p-test

Iterate through the primes and count targets

In [98]:
all_stats = pd.DataFrame()
for idx, nr in enumerate(skills):
    print(nr)
    turns_with_skill = set(np.where(hits[:,idx])[0])
    antecedent_skills = np.vstack([hits[turn_attention[turn_nr],:].any(axis=0) for turn_nr in turns_with_skill])
    other_skills = np.vstack([hits[turn_attention[turn_nr],:].any(axis=0) for turn_nr in range(len(turns)) if turn_nr not in turns_with_skill and turn_nr+2 not in turns_with_skill])
    
    x1 = antecedent_skills.sum(axis=0)
    n1 = len(antecedent_skills)
    p1 = x1 / n1
    x2 = other_skills.sum(axis=0)
    n2 = len(other_skills)
    p2 = x2 / n2
    
    stats = pd.DataFrame({'prime': nr, "target": skills,
                          "x1":x1, "n1":n1, "p1":p1,
                          "x2":x2, "n2":n2, "p2":p2,
                          "p1-p2": p1-p2})
    all_stats = pd.concat([all_stats, stats])

    #p_pool = (x1 + x2) / (n1 + n2)
    #SE = np.sqrt(p_pool * (1 - p_pool) * (1/n1 + 1/n2))
    #z = (p1 - p2) / SE
    #p = p_val(z)
    indices = np.where((p1-p2)>0.05)
    print(list(zip(np.array(skills)[indices], (p1-p2)[indices])))

57
[]
58
[]
59
[(619, 0.05517151756404602)]
69
[]
76
[(619, 0.08036823275103122)]
77
[(619, 0.05790910283707562)]
616
[(618, 0.06882724174979478), (619, 0.0505452864752504), (621, 0.05040137953874835)]
618
[(616, 0.05976340463734406), (618, 0.09141164875521098), (619, 0.09251231866726402)]
619
[(619, 0.11644785777734987)]
621
[(616, 0.13242198222524254), (618, 0.13460310497978145), (619, 0.1952559181658533)]
624
[(619, 0.12394187955191875)]
625
[(619, 0.16237630283761123)]
628
[(619, 0.19008928435543956)]
629
[(619, 0.11913369409137246)]
630
[(619, 0.12279070214630791)]
631
[(619, 0.11236684982459001)]
634
[(619, 0.06887603078948645)]
635
[(619, 0.08710085033340675), (625, 0.05209106972507497), (1176, 0.06339578735268539)]
636
[(619, 0.1110765530599654)]
1106
[]
1112
[(619, 0.17973786463180114)]
1116
[(625, 0.12868330059657274), (631, 0.11016558893300175), (1116, 0.09207586423835384), (1175, 0.13131698455446528), (1179, 0.11341397519703539)]
1175
[]
1176
[]
1177
[]
1178
[]
1179
[(1179,

Test for significant differences with Fisher's exact test

In [99]:
def get_p_value(row):
    table=[[row['x1'], row['n1']-row['x1']],
           [row['x2'], row['n2']-row['x2']]]
    odds_ratio, p_value = sps.fisher_exact(table, "greater")
    return pd.Series({'OR': odds_ratio, 'p': p_value})

all_stats[['OR', 'p']] = all_stats.apply(get_p_value, axis=1)

In [100]:
all_stats.reset_index(drop=True).to_json(primed_file)

In [101]:
all_stats = pd.read_json(primed_file)

In [102]:
alpha = 0.05 / len(all_stats)
print(alpha)

5.202913631633715e-05


In [104]:
all_stats[(all_stats['prime']==all_stats['target'])&(all_stats['p']<alpha)]

,prime,target,x1,n1,p1,x2,n2,p2,p1-p2,OR,p
0,57,57,7,209,0.033493,324,710230,0.000456,0.033037,75.928096,0.000000e+00
32,58,58,410,7643,0.053644,11956,695745,0.017184,0.036459,3.241915,8.541237e-85
64,59,59,103,3147,0.032730,4991,704448,0.007085,0.025645,4.742049,1.343990e-35
96,69,69,53,2774,0.019106,4530,705171,0.006424,0.012682,3.012622,0.000000e+00
128,76,76,18,831,0.021661,1357,708986,0.001914,0.019747,11.545367,0.000000e+00
192,616,616,274,6673,0.041061,9075,697625,0.013008,0.028053,3.248832,5.509896e-58
224,618,618,926,8702,0.106412,10407,693769,0.015001,0.091412,7.819518,0.000000e+00
256,619,619,7839,39671,0.197600,51632,636235,0.081152,0.116448,2.788296,0.000000e+00
288,621,621,33,2820,0.011702,3686,705198,0.005227,0.006475,2.253496,2.861360e-05
320,624,624,159,3595,0.044228,5689,703555,0.008086,0.036142,5.676493,1.960701e-63


In [92]:
all_stats[all_stats['p']<alpha].sort_values("OR", ascending=False)

5.202913631633715e-05


,index,prime,target,x1,n1,p1,x2,n2,p2,SE,p,p1-p2,OR
672,21,1116,1116,16,173,0.092486,291,710302,0.000410,0.001580,1.405981e-32,0.092076,248.652264
0,0,57,57,7,209,0.033493,324,710230,0.000456,0.001493,1.287910e-11,0.033037,75.928096
666,15,1116,631,20,173,0.115607,3865,710302,0.005441,0.005607,1.860484e-20,0.110166,23.892550
486,21,631,1116,18,2054,0.008763,286,706573,0.000405,0.000458,4.084911e-18,0.008359,21.832824
362,21,625,1116,30,3865,0.007762,275,703023,0.000391,0.000335,8.352642e-28,0.007371,19.990461
...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,26,619,1179,2234,39671,0.056313,30165,636235,0.047412,0.001105,2.096150e-15,0.008901,1.198951
54,23,58,1176,689,7643,0.090148,53108,695745,0.076333,0.003057,5.631980e-06,0.013815,1.198920
276,28,619,1192,2140,39671,0.053944,29318,636235,0.046080,0.001090,9.283968e-13,0.007863,1.180371
275,27,619,1187,784,39671,0.019763,10807,636235,0.016986,0.000672,2.806567e-05,0.002777,1.166766


## Establish causality by simulating an intervention

Now let's use one of them in a random dialog context and simulate the next answer.

In [23]:
model, tokenizer = models.load_generator("meta-llama/Meta-Llama-3-8B-Instruct")#"/cluster/home/dglandorf/models/llama-FT")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Sample random contexts and override the constraint

In [24]:
n = 64
nr = 621
cases = pd.DataFrame([helpers.sample_dialog_snippet(dialogs) for _ in range(n)])
cases.columns = ['context','response','source','id']
cases['constraints']=[[nr]] * n
cases = cases.apply(lambda x: helpers.get_generation_prompt(x, tokenizer.apply_chat_template, system_msg=True), axis=1)

Generate constrained answers

In [23]:
cases['response'] = models.generate(model, tokenizer, list(cases['prompt']), verbose=False, do_sample=False, eos_token_id=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")], batch_size=8)

Generate: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:02<00:00,  7.83s/it]


Append the answers to the context that fulfill the constraint and create prompts for an unconstrained answer to simulate the learner

In [24]:
print(len(cases))
cases = cases[(models.probe_model(classifiers[nr], list(cases['response']))[0] > 0.5).numpy()]
print(len(cases))

64
63


In [25]:
cases['context'] = cases.apply(lambda x: x['context'] + [x['response'].replace("A: ", "")], axis=1)
cases = cases.apply(lambda x: helpers.get_generation_prompt(x, tokenizer.apply_chat_template, system_msg=True, unconstrained=True), axis=1)

Generate next response again

In [27]:
cases['response'] = models.generate(model, tokenizer, list(cases['prompt']), verbose=False, do_sample=False, eos_token_id=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")], batch_size=8)

Generate: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:19<00:00,  9.94s/it]


In [30]:
cases['context'] = cases.apply(lambda x: x['context'] + [x['response'].replace("B: ", "")], axis=1)

In [34]:
cases['context'].sample(1).iloc[0]

['Yes that was until 1952 along with sculpture, music and painting',
 'Yeah. DO you go to the library much?',
 'I go sometimes. Did you hear about that guy in prison that proved his own innocence?',
 'Yes he studies at the library and then also helped other people',
 'Would you like to hear more about that story?',
 "Here is a possible next turn of B:\n\nB: That sounds really interesting, I'd love to hear more about it."]

Test for target structures

In [35]:
for nr in primed[nr]:
    print(nr)
    print((models.probe_model(classifiers[nr], list(cases['response']))[0]>0.5).float().mean())

616
tensor(0.0635)
618
tensor(0.0794)
619
tensor(0.6667)


Now as a loop for all primes

In [41]:
n=32


In [25]:
def generate_responses(cases):
    cases['response'] = models.generate(model, tokenizer, list(cases['prompt']), verbose=False, do_sample=False, eos_token_id=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")], batch_size=8)

In [26]:
def sample_succesful(constraint, n=8):
    cases = pd.DataFrame([helpers.sample_dialog_snippet(dialogs) for _ in range(n)])
    cases.columns = ['context','response','source','id']
    cases['constraints'] = [[constraint]] * n
    cases = cases.apply(lambda x: helpers.get_generation_prompt(x, tokenizer.apply_chat_template, system_msg=True), axis=1)
    generate_responses(cases)
    success = (models.probe_model(classifiers[constraint], list(cases['response']))[0] > 0.5).numpy()
    print(success.mean())
    return cases[success]

In [30]:
ratios = {}
for prime_nr, target_nrs in primed.items():
    print(prime_nr)
    if not len(target_nrs): continue
    ratios[prime_nr] = {}
    cases = pd.DataFrame()
    attempts = 0
    while len(cases) < 32 and attempts < 16:
        cases = pd.concat([cases, sample_succesful(prime_nr)], ignore_index=True)
        attempts += 1
    if not len(cases): continue
    cases['context'] = cases.apply(lambda x: x['context'] + [x['response'].replace("A: ", "")], axis=1)
    print(cases['context'].sample(1).iloc[0])
    cases = cases.apply(lambda x: helpers.get_generation_prompt(x, tokenizer.apply_chat_template, unconstrained=True, system_msg=True), axis=1)
    cases['response'] = models.generate(model, tokenizer, list(cases['prompt']), verbose=False, do_sample=False, eos_token_id=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")], batch_size=8)
    for nr in target_nrs:
        print(nr)
        ratios[prime_nr][nr] = (models.probe_model(classifiers[nr], list(cases['response']))[0]>0.5).float().mean()
        print(ratios[prime_nr][nr])

57
58
59


Generate: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.30s/it]


0.375


Generate: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.16s/it]


0.375


...

0.625


Generate: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.54s/it]


0.75
["That's nice. I'd like to give it a try. I think I like passion red. please.", 'Alright.Do you want your cuticles cut, too?', 'No.', 'Would you like the shapes square or round?', "I didn't think so."]


Generate: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:37<00:00,  7.45s/it]

1179
tensor(0.2424)
1187
1192
1197
1198
